# Un ejemplo de proceso de puntos (point pattern)

En **geoestadística** y en **econometría espacial**, se considera el espacio como un dominio en el que a cada punto del espacio le corresponde un atributo, siendo las localizaciones de los objetos espaciales fijas y donde se espera que "todas" las localizaciones tengan un valor.   
  -  La Geoestadística trata fundamentalmente de hacer predicciones sobre el valor de un atributo en un punto no observado utilizando la información de otros puntos donde si se ha muestreado información sobre dicho atributo. Por ejemplo datos de precipitaciones o de presión atmosférica, o de contaminación. Se hace un muestreo de los datos de en algunos puntos del espacio (donde existen estaciones meteorilógicas o de medición), y se busca **interpolar** esos datos a puntos intermedios donde no se dispone de datos.
  -  En Econometría normalmente se trabajan con polígonos, es decir, los datos espaciales son en realidad, polígonos que representan diferentes estructural geopolíticas, como municipios, países, distritos, provincias, códigos postales, y se tiene información sobre todos ellos. EL objetivo en econometría espacial no es tanto la interpolación para predecir valores como la **modelización de la posible dependencia o aoutocorrelación espacial**: relación o dependencia del valor de un atributo en una regíon respecto al valor que toma ese mismo atributo en sus vecinos más cercanos.
  
   También es posible trabajar con modelos de regresión espacial con datos de puntos, por ejemplo precios de la vivienda, pero nuevamente el objetivo es modelizar la depedencia espacial, más que la interpolaión (aunque claramente aquí es donde geoestadísitca y regresión espacial acaban convergiendo, ya que en ambos tipos de modelización, por vías diferentes se intenta alcanzar el mismo objetivo, predecir el valor de un avivienda).

  - En ocasiones nos enfrentamos a datos espaciales donde los propios puntos en el espacio para los que se tiene información són el objeto de estudio. En estos casos los puntos observados se consideran sucesos que podrían tener lugar en varios lugares pero que sólo ocurren en unos pocos, una colección de tales sucesos se denomina **proceso de puntos o point pattern.** Un excelente resumen de este tipo de datos puede encontrarse en el libro de [Sergio Rey et al. Geographic Data Science with Python](https://geographicdata.science/book/notebooks/08_point_pattern_analysis.html)


Un ejemplo de estos procesos de puntos donde a localización de los puntos es uno de los aspectos clave de interés para el análisis es por ejemplo la ubicación de las recogidas de clientes de UBER . En esta práctica voy a utilizar los datos de recogidas de [UBER en la ciudad de Nueva York que están disponibles en Kagle](https://www.kaggle.com/datasets/tekbahadurkshetri/uber-clustering). Para vuestra comodidad os he dejado los datos en el archivo "uber_clean.csv".

Con este tipo de procesos de puntos puede ser interesante analizar la propia **frecuencia de localizaciones donde aparecen los datos**, para detectar, por ejemplo, puntos calientes (¿por qué los eventos ocurren en algunal localizaciones y no en otras?), y para comprobar si hay algún **patrón espacial en la localización de los eventos**, o si por el contrario puede suponerse una **distribución espacial de los puntos completamente aleatoria**.

Los patrones de puntos pueden estar **marcados**, si se proporcionan más atributos con la ubicación, o **sin marcar**, si sólo se proporcionan las coordenadas del lugar donde se produjo el suceso.

Nosotros utilizaremos los datos también para realizar un análisis de **cluster o agrupación de datos espaciales** con el objetivo de poder dividir la ciudad de Nueva York en diferentes subzonas de negocio



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.cluster import dbscan

from sklearn.metrics import silhouette_score

import contextily


import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd


from pointpats import centrography

#conda install -c conda-forge folium
import folium



Leo los datos desde el archivo 'uber_clean.csv', de la carpeta datos, 

In [ ]:
df = pd.read_csv('datos/uber_clean.csv')  
print(df.shape)
print(df.info())
df.head()




Fíjate que en este conjunto de datos sólo se dispone de información del punto de recogida de los clientes, por lo que trataremos los datos como un proceso de puntos no marcados (la información de la base la utilizaremos posteriormente para comprobar la bondad de nuestro proceso de agrupación de datos).    
     

Para que dure algo menos de tiempo la práctica voy a hacer una subselección de sólo 50.000 registros. Vosotros podéis intentar realizar el mismo análisis con el conjunto completo de datos.


In [ ]:
df=df.sample(n=50000, frac=None, replace=False, weights=None, random_state=1234)
df.shape

# Visualización de patrones puntuales 

In [ ]:
plt.scatter(x='Lon', y='Lat',data=df, alpha=0.5, s=1)

In [ ]:
# con seaborn
sns.jointplot(x="Lon", y="Lat", data=df, s=0.5)



Para dar cierto contexto geográfico podemos por ejemplo utilizar la **librería contextily**

In [ ]:
joint_axes = sns.jointplot(
    x="Lon", y="Lat", data=df, s=0.5
)
contextily.add_basemap(
    joint_axes.ax_joint,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
)

También podríamos utilizar mapas dinámicos con leaflet utilizando la librería folium. Que permite realizar de forma sencilla una exploración del mapa de Open StreetMap    
    
Primero seleccione el mapa sobre el que voy a representar los puntos

In [ ]:
map = folium.Map(location=[40.7128, -74.0060], zoom_start=10,tiles = "openstreetmap")
map

Ahora represento los puntos en el mapa anterior

In [ ]:
# for i in range(len(df)):
# como son muchos puntos, solo voy a mostrar los 2000 primeros
for i in range(0,2000):
    folium.CircleMarker(
        location=[df.iloc[i]['Lat'], df.iloc[i]['Lon']],
        radius=5,
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(map)

map

# Mapa de densidad de frecuencias

Existen zonas donde la densidad de puntos es tan alta que sólo se ve algo realizando un zoom. Para estos casos resulta muy util hacer un histograma de de frecuencias utilizando una función de densidad para la longitud y la latitud (mapas de calor), conocidos como histogramas 2-dimensionales o espaciales.

La construcción de estos histogramas espaciales se realiza estableciendo una cuadrícula regular (cuadrada o hexagonal) sobre el mapa y posteriormente contando cuántos puntos hay en cada celda de la cuadrícula. Después se presentan los recuentos como lo haríamos con cualquier otro mapa de coroplóticos o de coropletas. 

Aquí utilizo la tramificación o binning hexagonal (a veces llamado hexbin) porque tiene propiedades ligeramente mejores que las cuadrículas cuadradas, como una menor distorsión de la forma y una conectividad más regular entre las celdas.

In [ ]:
# inicializamos el gráfico
f, ax = plt.subplots(1, figsize=(12, 9))
# Generamos y añadimos la retícula hexagonal con 500 hexagononos
hb = ax.hexbin(
    df["Lon"],
    df["Lat"],
    gridsize=500,
    linewidths=0,
    alpha=0.5,
    cmap="viridis_r",
)
# añadimos el mapa base
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels,
    )
# añadimos la barra de color
plt.colorbar(hb)


Hacemos un zoom

In [ ]:
#filtro para utilizar sólo datos de la area de Manhattan
df_filtered = df[(df['Lon'] >= -74.05) & (df['Lon'] <= -73.85) & (df['Lat'] >= 40.66) & (df['Lat'] <= 40.82)]
# Set up figure and axis
f, ax = plt.subplots(1, figsize=(12, 9))
# Generamos y añadimos la retícula hexagonal con 50 hexagononos
hb = ax.hexbin(
    df_filtered["Lon"],
    df_filtered["Lat"],
    gridsize=50,
    linewidths=0,
    alpha=0.5,
    cmap="viridis_r",
)
# añadimos el mapa base
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels,
    )
# añadimos la barra de color
plt.colorbar(hb)
# eliminamos los ejes
#ax.set_axis_off()



También podemos hacer el histograma espacial utilizando una **función de densidad Kernel**

In [ ]:
#filtro para utilizar sólo datos de la area de Manhattan
# df_filtered = df[(df['Lon'] >= -74.05) & (df['Lon'] <= -73.85) & (df['Lat'] >= 40.66) & (df['Lat'] <= 40.82)]

f, ax = plt.subplots(1, figsize=(9, 9))
# Generamos la función de densidad de kernel
sns.kdeplot(
    x="Lon",
    y="Lat",
    data=df_filtered,
    n_levels=100,
    fill=True,
    alpha=0.5,
    cmap="viridis_r",
)
# añadimos el mapa base
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
    )


# Centrografía

La centrografía es el análisis de la centralidad en un patrón de puntos. Por **centralidad** entendemos la ubicación general y la dispersión del patrón. Si el hexágono anterior puede verse como un **histograma espacial**, la centrografía es el equivalente en patrones puntuales de las **medidas de tendencia central, como la media**.    
    
Estas medidas son útiles porque nos permiten **resumir las distribuciones espaciales** en conjuntos de información más pequeños (por ejemplo, un solo punto). En la centrografía se utilizan muchos índices diferentes para proporcionar una indicación de **dónde** se encuentra un patrón de puntos, la **intensidad** con la que el patrón de puntos se agrupa en torno a su centro o lo **irregular de su forma**.

## Medida de tendencia central: "centro de masa" 


Para patrones puntuales no marcados esta tendencia central o centro de masa sería la media de las coordenadas; para patrones de puntos markados sería el punto central de los datos con mayor valor de su atributo marcado

In [ ]:
# medida de tendencia central: "centro de masa" (para patrones puntuales no marcados sería la media de las coordenadas, para patrones de puntos markados sería el punto central de los datos con mayor valor de su atributo marcado)
 
from pointpats import centrography

mean_center = centrography.mean_center(df[["Lon", "Lat"]])
mediana_center = centrography.euclidean_median(df[["Lon", "Lat"]])

print(mean_center, mediana_center)

In [ ]:
# Voy a representar la tendencia central
joint_axes = sns.jointplot(
    x="Lon", y="Lat", data=df, s=0.75, height=9
)
# Añado punto medio
joint_axes.ax_joint.scatter(
    *mean_center, color="red", marker="x", s=50, label="Centro Medio"
)
joint_axes.ax_marg_x.axvline(mean_center[0], color="red")
joint_axes.ax_marg_y.axhline(mean_center[1], color="red")
# Añado punto mediano
joint_axes.ax_joint.scatter(
    *mediana_center,
    color="limegreen",
    marker="o",
    s=50,
    label="Centro Mediano"
)
joint_axes.ax_marg_x.axvline(mediana_center[0], color="limegreen")
joint_axes.ax_marg_y.axhline(mediana_center[1], color="limegreen")
# Legend
joint_axes.ax_joint.legend()
# añadimos el mapa base
contextily.add_basemap(
    joint_axes.ax_joint,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
)
# Clean axes
joint_axes.ax_joint.set_axis_off()
# Display
plt.show()

In [ ]:
# Lo hago sólo para el centro de Manhattan
mean_center = centrography.mean_center(df_filtered[["Lon", "Lat"]])
mediana_center = centrography.euclidean_median(df_filtered[["Lon", "Lat"]])

print(mean_center, mediana_center)

In [ ]:
# Voy a representar la tendencia central
joint_axes = sns.jointplot(
    x="Lon", y="Lat", data=df_filtered, s=0.75, height=9
)
# Añado punto medio
joint_axes.ax_joint.scatter(
    *mean_center, color="red", marker="x", s=50, label="Centro Medio"
)
joint_axes.ax_marg_x.axvline(mean_center[0], color="red")
joint_axes.ax_marg_y.axhline(mean_center[1], color="red")
# Añado punto mediano
joint_axes.ax_joint.scatter(
    *mediana_center,
    color="limegreen",
    marker="o",
    s=50,
    label="Centro Mediano"
)
joint_axes.ax_marg_x.axvline(mediana_center[0], color="limegreen")
joint_axes.ax_marg_y.axhline(mediana_center[1], color="limegreen")
# Legend
joint_axes.ax_joint.legend()
# añadimos el mapa base
contextily.add_basemap(
    joint_axes.ax_joint,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
)
# Clean axes
joint_axes.ax_joint.set_axis_off()
# Display
plt.show()

In [ ]:
# Lo hago ahora con el grtáfico de densidad

    
f, ax = plt.subplots(1, figsize=(9, 9))
# Generamos la función de densidad de kernel
sns.kdeplot(
    x="Lon",
    y="Lat",
    data=df_filtered,
    n_levels=100,
    fill=True,
    alpha=0.5,
    cmap="viridis_r",
)
# Añado punto medio
ax.scatter(
    *mean_center, color="red", marker="x", s=50, label="Centro Medio"
)
# Añado punto mediano
    
ax.scatter(
    *mediana_center,
    color="limegreen",
    marker="o",
    s=50,
    label="Centro Mediano"
)
# añadimos el mapa base
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
    )
# añadimos la leyenda
ax.legend()
# eliminamos los ejes
ax.set_axis_off()
# Display
plt.show()
    

## Medida de dispersión: "distancia típica" (standard distance)

Esta medida proporciona la distancia media al centro de la nube de puntos (como la medida por el centro de masa)

In [ ]:
centrography.std_distance(df[["Lon", "Lat"]])

Esto significa que, por término medio, los uber se toman a unos 0.0726 grados equivalente a unos (0.0726*40000/360)~=8 kilómetros del centro medio.

Para visualizar esta "distancia típica" se utiliza la **elipse típica o la elipse de desviación típica** 

In [ ]:
major, minor, rotation = centrography.ellipse(df_filtered[["Lon", "Lat"]])

from matplotlib.patches import Ellipse

# prerparo el gráfico
f, ax = plt.subplots(1, figsize=(9, 9))
# dibujo pos pontos t los centros de masa
ax.scatter(df_filtered["Lon"], df_filtered["Lat"], s=0.75)
ax.scatter(*mean_center, color="red", marker="x", label="Centro Medio")
ax.scatter(
    *mediana_center, color="limegreen", marker="o", label="Centro Mediano"
)

# creo la elipse
ellipse = Ellipse(
    xy=mean_center,  # center the ellipse on our mean center
    width=major * 2,  # centrography.ellipse only gives half the axis
    height=minor * 2,
    angle=np.rad2deg(
        rotation
    ),  # Angles for this are in degrees, not radians
    facecolor="none",
    edgecolor="red",
    linestyle="--",
    label="Std. Ellipse",
)
ax.add_patch(ellipse)

ax.legend()
# añadimos el mapa base
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
    )
plt.show()



## Delimitación de áreas

Ahora vamos a hacer un ejercicio de delimitación de áreas. Fíjate que dentro del área de Manhattan hay actuando UBERs que pertenecen a CincoBases 

In [ ]:
frequencies = df_filtered['Base'].value_counts()
print(frequencies)

Voy a representarlas con diferentes colores


In [ ]:
# prerparo el gráfico
plt.figure(figsize=(10, 6))
# dibujo pos pontos t los centros de masa
ax=sns.scatterplot(x='Lon', y='Lat', hue='Base', data=df_filtered, palette='tab10', s=10)
ax.legend()
# añadimos el mapa base
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
    )

plt.title('recogidas según Base de UBER')






Voy a intentar delimitar el área de actuación de los uber de la base B02682

In [31]:
user = df_filtered[df_filtered['Base'] == "B02682"]
coordinates = user[["Lon", "Lat"]].values


**Convex hull**     
En primer lugar, calcularemos el casco convexo (**covex hull**), que es la forma convexa más cerrada que encierra todos los viajes de UBER de la Base B028682. Por convexa, entendemos que la forma nunca se «dobla» sobre sí misma; no tiene divisiones, valles, crenulaciones ni agujeros.

In [32]:
convex_hull_vertices = centrography.hull(coordinates)

In [ ]:

# prerparo el gráfico
f, ax = plt.subplots(1, figsize=(9, 9))

# Representar los datos de coordinates seleccionados
ax.scatter(coordinates[:, 0], coordinates[:, 1], s=1, c='red', label='Datos B02682')

# Representar el polígono del convex_hull_vertices
hull_polygon = plt.Polygon(convex_hull_vertices, edgecolor='green', fill=None, label='Convex Hull')

# Añadir el polígono al gráfico
ax.add_patch(hull_polygon)

# Añadir leyenda
plt.legend()

# añadimos el mapa base
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
    )



# Añadir título y etiquetas
plt.title('Representación de datos y Convex Hull')
plt.xlabel('Lon')
plt.ylabel('Lat')

# Mostrar el gráfico
plt.show()

**Alpha shape**    
En segundo lugar, puede calcularse la denominada **forma alfa** ((alpha shape)), que puede entenderse como una versión *más ajustada* del casco convexo. Una forma de pensar en una forma alfa es como el espacio creado al rodar una pelota pequeña alrededor de la forma. Como la pelota es más pequeña, rueda en los desniveles y valles creados entre los puntos. A medida que la pelota crece, la forma alfa se convierte en el casco convexo. Pero, para las bolas pequeñas, la forma puede ser muy estrecha. De hecho, si alfa se hace demasiado pequeña, se «desliza» por los puntos, ¡dando lugar a más de un casco!


In [40]:
import libpysal

alpha_shape, alpha, circs = libpysal.cg.alpha_shape_auto(
    coordinates, return_circles=True
)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(9, 9))

# Plot a green alpha shape
gpd.GeoSeries(
    [alpha_shape]
).plot(
    ax=ax,
    edgecolor="green",
    facecolor="green",
    alpha=0.2,
    label="Forma alfa",
)

# Dibujamos los puntos 
ax.scatter(
    *coordinates.T, color="k", marker=".", label="Datos B02682"
)

# dibujamos los círculos que forman la frontera de la forma alpha 
for i, circle in enumerate(circs):
    # sólo etiquetamos el primero de los círculos
    if i == 0:
        label = "Bounding Circles"
    else:
        label = None
        ax.add_patch(
            plt.Circle(
                circle,
                radius=alpha,
                facecolor="none",
                edgecolor="r",
                label=label,
            )
        )
        
# añadimos convex hull
ax.add_patch(
    plt.Polygon(
        convex_hull_vertices,
        closed=True,
        edgecolor="blue",
        facecolor="none",
        linestyle=":",
        linewidth=2,
        label="Convex Hull",
    )
)

## añadimos el mapa base
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
    )



plt.legend();

 - Dejo como ejercicio que representéis conjuntamente el convex Hull ( y el alpha shape) de las cinco bases para detenctar diferencias en las áreas de actuación de cada una de ellas.     


- Otro ejercicio podría ser intentar detectar diferencias tanto en la distribución de puntos (gráfico de densidad) y en área de actuación de los días entre diario y de los fines de semana (¿se aprecia alguna diferencia?)

# Aleatoriedad y concentración espacial 

Uno de los objetivos del análisis de procesos de puntos es averiguar si los eventos aparecen de forma totalmente aleatoria sobre el espacio o si por el contrario existe cierto patrón espacial que haga que los eventos sucedan unos cerca de otros, formando concnetraciones de eventos, o agrupamiento de eventos. Cuando vamos los modelos de regresión espacial volveremos sobre esta idea de distribución espacial, y de la existencia de patrones de comportamiento espacial. Ahora hablamos de patrones espaciales en la localización de eventos, y cuando pasemos a los modelos de autocorrelación espacial hablaremos de dependencia espacial de los valores de un atributo respecto a los valores de ese atributo en las regiones vecinas.    

 
Para detectar estos patrones espaciales existen diferentes téncinas. El primer conjunto de técnicas se denominan **estadísticas de cuadrantes** y reciben su nombre por su planteamiento de dividir los datos en pequeñas áreas (cuadrantes). Una vez creados, estos «cubos» se utilizan para examinar la uniformidad de los recuentos en ellos. El segundo conjunto de técnicas procede de Ripley (1988) y consiste en medir la distancia entre puntos en un patrón de puntos.


A efectos comparativos, resulta útil proporcionar el patrón derivado de un proceso completamente aleatorio desde el punto de vista espacial. Es decir, la ubicación y el número de puntos son totalmente aleatorios; no hay ni agrupación ni dispersión. En el análisis de patrones de puntos, esto se conoce como proceso de puntos de Poisson.



In [43]:
from pointpats import (
    distance_statistics,
    QStatistic,
    random,
    PointPattern,
)

In [44]:
# simulación de datos aleatorios en relación a los datos de la base B02682 (que hemos almacenado en "coordinates")
random_pattern = random.poisson(coordinates, size=len(coordinates))

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
plt.scatter(
    *coordinates.T,
    color="k",
    marker=".",
    label="datos de la base B02682 "
)
plt.scatter(*random_pattern.T, color="r", marker="x", alpha=0.5, label="Proceso Aleatorio Espacial")
## añadimos el mapa base
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
    )
ax.legend(ncol=1, loc="upper left")
plt.show()

La distribución aleatoria se ha hecho por defecto en una frontera cuadrada, si quisiesemos utilizar otro tipo de forma (convex hull o alpha shape) también podríamos hacerlo

In [47]:
random_pattern_ashape = random.poisson(
    alpha_shape, size=len(coordinates)
)

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
plt.scatter(
    *coordinates.T,
    color="k",
    marker=".",
    label="datos de la base B02682 "
)
plt.scatter(*random_pattern_ashape.T, color="r", marker="x", alpha=0.5, label="Proceso Aleatorio Espacial")
## añadimos el mapa base
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
    )
ax.legend(ncol=1, loc="upper left")
plt.show()

## Quadrat statistics

Las estadísticas de cuadrantes (quadrat statistics) analizan la distribución espacial de los puntos en función del recuento de observaciones que caen dentro de una celda determinada. Al examinar si las observaciones se distribuyen uniformemente por el conjunto de celdas, el enfoque de cuadrantes pretende estimar si los puntos están dispersos o si están agrupados en unas pocas celdas. En sentido estricto, la estadística de cuadrantes examina la uniformidad de la distribución en las celdas mediante una prueba estadística X2 habitual en el análisis de contingencias. 

In [ ]:
qstat = QStatistic(coordinates)
qstat.plot()


In [ ]:
# p-valor del test de Chi2 para la Ho de distribución uniforme entre todos los cuadrantes
qstat.chi2_pvalue

Para comparar vemos los resultados con el proceso totalmente aleatorio que hemos estimado anteriormente

In [ ]:
qstat_null = QStatistic(random_pattern)
qstat_null.plot()

In [ ]:
qstat_null.chi2_pvalue

El problema con este enfoque es que supone que todos los puntos en el cuadrado son susceptibles de ubicar un evento del proceso. Y como es claro en nuestro caso, los Uber no pueden recoger a clientes en el mar. Por eso hay que seguir analizando otro tipo de técnicas.    

Igualmente, el hecho de que los recuentos de cuadrantes se midan en un mosaico regular que se superpone a la extensión potencialmente irregular de nuestro patrón puede inducirnos a error. En particular, los patrones irregulares pero aleatorios pueden ser erróneamente considerados «significativos» por este enfoque. Consideremos nuestro conjunto aleatorio generado dentro del polígono de forma alfa con la cuadrícula de cuadrantes superpuesta


In [ ]:
qstat_null_ashape = QStatistic(random_pattern_ashape)
qstat_null_ashape.plot()

In [ ]:
qstat_null_ashape.chi2_pvalue

Nótese que estamos rechazando la hipótesis nual de aleatoriedad, auqnue sabemos que por contrcción la distribución es totalmente aleatoria en el alpha shape. En conclusión este tipo de estadísticas de cuadrantes sólo puede considerarse como válidads en supoerficies regulares

## Ripley’s alphabet of functions

El segundo grupo de estadísticas espaciales que consideramos se centra en las distribuciones de dos cantidades en un patrón de puntos: las distancias a los vecinos más próximos y lo que denominaremos «huecos» en el patrón. Se derivan del trabajo seminal de Ripley(1991) sobre cómo caracterizar la agrupación o co-localización en patrones de puntos. Cada una de ellas caracteriza un aspecto del patrón de puntos a medida que aumentamos el rango de distancia desde cada punto para calcularlas.    

(Brian D Ripley. Statistical inference for spatial processes. Cambridge University Press, 1991.)

La primera función de Ripley se centra en la distribución de las **distancias a los vecinos más próximos**. Esta función resume las distancias entre cada punto y su vecino más cercano. **La G de Ripley** calcula la proporción de puntos en los que el vecino más próximo se encuentra dentro de un umbral de distancia determinado y traza el porcentaje acumulado en función de los radios de distancia crecientes. La distribución de estos porcentajes acumulativos tiene una forma característica en procesos completamente aleatorios desde el punto de vista espacial.     
      
La intuición que subyace a la G de Ripley es la siguiente: podemos saber hasta qué punto nuestro patrón se parece a uno espacialmente aleatorio calculando la distribución acumulativa de las distancias a los vecinos más próximos sobre umbrales de distancia crecientes y comparándola con la de un conjunto de patrones simulados que siguen un proceso espacialmente aleatorio conocido. Normalmente, se utiliza un proceso espacial de Poisson como distribución de referencia.

In [ ]:
coordinates.shape


In [ ]:
np.random.seed(1234)
coordinates_sample = coordinates[np.random.choice(coordinates.shape[0], 500, replace=False)]

In [71]:
# de la librería pointpats 
# como tarda mucho (son más de 11 mil localizaciones las que tiene que analizar) voy a hacer una selección aleatoria sólo de 500 puntos 
np.random.seed(1234)
coordinates=coordinates[np.random.choice(coordinates.shape[0], 500, replace=False)]

g_test = distance_statistics.g_test(
    coordinates, support=40, keep_simulations=True
)

Pensando en estas distribuciones de distancias, un patrón «agrupado» debe tener más puntos cercanos entre sí que un patrón «disperso»; y un patrón completamente aleatorio debería tener algo intermedio. Por lo tanto, si la función aumenta rápidamente con la distancia, probablemente estemos ante un patrón agrupado. Si aumenta lentamente con la distancia, tenemos un patrón disperso. Algo intermedio será difícil de distinguir del puro azar.

In [ ]:
f, ax = plt.subplots(
    1, 2, figsize=(9, 3), gridspec_kw=dict(width_ratios=(6, 3))
)
# plot all the simulations with very fine lines
ax[0].plot(
    g_test.support, g_test.simulations.T, color="k", alpha=0.01
)
# and show the average of simulations
ax[0].plot(
    g_test.support,
    np.median(g_test.simulations, axis=0),
    color="cyan",
    label="median simulation",
)


# and the observed pattern's G function
ax[0].plot(
    g_test.support, g_test.statistic, label="observed", color="red"
)

# clean up labels and axes
ax[0].set_xlabel("distance")
ax[0].set_ylabel("% of nearest neighbor\ndistances shorter")
ax[0].legend()
ax[0].set_title(r"Ripley's $G(d)$ function")

# plot the pattern itself on the next frame
ax[1].scatter(*coordinates.T,marker=".", s=0.5)
contextily.add_basemap(
    ax[1],
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
    )

# and clean up labels and axes there, too
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].set_xticklabels([])
ax[1].set_yticklabels([])
ax[1].set_title("Pattern")
f.tight_layout()
plt.show()

En este caso, el gráfico muestra un crecimiento del porcentaje acumulado de vecinos más rápido que el correspondiente a una distirbución aleatoria, por lo que podemos concluir que existen patrones espaciales de concentración de eventos.Existen otras técinas y funciones (como la función F de Ripley), pero que quedal fuera del alcance de esta clase. véase [Rey et al , 2023](https://geographicdata.science/book/notebooks/08_point_pattern_analysis.html#centrography)

## Identificación de las agrupación o Clustering (DBSCAN)      
Las dos secciones anteriores sobre el análisis espacial exploratorio de patrones de puntos proporcionan métodos para caracterizar si los patrones de puntos están dispersos o agrupados en el espacio. Otra forma de ver el contenido de esas secciones es que nos ayudan a explorar el grado de agrupación general. Sin embargo, saber que un patrón de puntos está agrupado no nos da necesariamente información sobre dónde reside ese (conjunto de) conglomerado(s). Para ello, necesitamos pasar a un método capaz de identificar zonas de alta densidad de puntos dentro de nuestro patrón. En otras palabras, en esta sección nos centramos en la existencia y localización de los clusters.

De los muchos algoritmos de agrupamiento espacial de puntos, vamos a utilizar el llamado DBSCAN (Density-Based Spatial Clustering of Applications) [EKS+96]. DBSCAN es un algoritmo ampliamente utilizado que se originó en el área de descubrimiento de conocimiento y aprendizaje automático y que desde entonces se ha extendido a muchas áreas, incluyendo el análisis de puntos espaciales.

Referencia: Martin Ester, Hans-Peter Kriegel, Jörg Sander, Xiaowei Xu, and others. A density-based algorithm for discovering clusters in large spatial databases with noise. In Kdd, volume 96, 226–231. 1996.

Desde el punto de vista de DBSCAN, un cluster es una concentración de al menos **m** puntos, cada uno de ellos a una distancia de **r** de al menos otro punto del cluster. Siguiendo esta definición, el algoritmo clasifica cada punto de nuestro patrón en tres categorías:

 - Ruido, para aquellos puntos fuera de un cluster.

 - Núcleos, para aquellos puntos dentro de un cluster con al menos **m** puntos en el cluster dentro de la distancia **r**.


 - Fronteras, para los puntos dentro de un clúster con menos de m puntos en el clúster dentro de la distancia r.     

La flexibilidad (pero también algunas de las limitaciones) del algoritmo residen en que tanto **m** como **r** deben ser especificados por el usuario antes de ejecutar DBSCAN. Se trata de un punto crítico, ya que su valor puede influir significativamente en el resultado final. Antes de profundizar en este tema, vamos a hacer un primer intento de calcular DBSCAN en Python:

In [ ]:
# Define DBSCAN

from sklearn.cluster import DBSCAN

clusterer = DBSCAN()
# Fit to our data
clusterer.fit(df_filtered[["Lon", "Lat"]])

El atributo labels_ tiene siempre la misma longitud que el número de puntos utilizados para ejecutar DBSCAN. Cada valor representa el índice del cluster al que pertenece un punto. Si el punto se clasifica como ruido, recibe un -1. Para facilitar las cosas más adelante, convirtamos las etiquetas en un objeto Serie que podamos indexar del mismo modo que nuestra colección de puntos:

In [84]:
lbls = pd.Series(clusterer.labels_, index=df_filtered.index)

In [ ]:
# ahora podemos visualizar los clusters
# Setup figure and axis

f, ax = plt.subplots(1, figsize=(9, 9))
# Subset points that are not part of any cluster (noise)
noise = df_filtered.loc[lbls == -1, ["Lon", "Lat"]]

# Plot noise in grey
ax.scatter(noise["Lon"], noise["Lat"], c="black", s=10, linewidth=0)

# Plot all points that are not noise in red
# NOTE how this is done through some fancy indexing, where
#      we take the index of all points (tw) and substract from
#      it the index of those that are noise
ax.scatter(
    df_filtered.loc[df_filtered.index.difference(noise.index), "Lon"],
    df_filtered.loc[df_filtered.index.difference(noise.index), "Lat"],
    c="red",
    marker=".", 
    s=0.5,
    linewidth=0,
)
# Add basemap
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
    )
# Remove axes
ax.set_axis_off()
# Display the figure
plt.show()

No parece muy satisfactorio, porque los valores por defecto de DBSCAN es un radio de 0.5 y m=5 puntos     

Vamos a cambiarlo para que el número de puntos sea un 1% del total de puntos. y que la distancia sea 0.5 veces la distancia típica

In [ ]:
# Obtain the number of points 5% of the total represents
minp = np.round(df_filtered.shape[0] * 0.05)
minp

In [ ]:
# 0.25, 0.5, o 0.75  veces la distancia típica
radio= centrography.std_distance(df_filtered[["Lon", "Lat"]])*0.5
radio

In [122]:
# Rerun DBSCAN
clusterer = DBSCAN(eps=radio, min_samples=int(minp))
clusterer.fit(df_filtered[["Lon", "Lat"]])

# Turn labels into a Series
lbls = pd.Series(clusterer.labels_, index=df_filtered.index)

In [ ]:


# Setup figure and axis
f, ax = plt.subplots(1, figsize=(9, 9))

# Subset points that are not part of any cluster (noise)
noise = df_filtered.loc[lbls == -1, ["Lon", "Lat"]]

# Plot noise in grey
ax.scatter(noise["Lon"], noise["Lat"], c="grey", s=5, linewidth=0)

# Plot all points that are not noise in red
# NOTE how this is done through some fancy indexing, where
#      we take the index of all points and substract from
#      it the index of those that are noise
ax.scatter(
    df_filtered.loc[df_filtered.index.difference(noise.index), "Lon"],
    df_filtered.loc[df_filtered.index.difference(noise.index), "Lat"],
    c="red",
     marker=".", 
    s=0.5,
    linewidth=0,
)
# Add basemap
contextily.add_basemap(
    ax,
    crs="EPSG:4326",  # hemos especificado el sistema de referencia internacional WGS84
    source=contextily.providers.CartoDB.PositronNoLabels
    )
# Remove axes
ax.set_axis_off()
# Display the figure
plt.show()

En gris aparecerían las ubicaciones que se consideran no agrupadas, y en rojo las que sí están agrupadas... podríamos seguir trabajando sólo con estas, pero de momento lo dejamos aquí, esto es, con la identificación de los puntos que pueden considerarse agrupados o concentrados según el patrón "desconocido" de distribución espacial de los puntos de recogida de UBER.

--------------------------------------